In [2]:
import os

import lucem_illud
import openai
import pandas as pd
from openai import OpenAI

# Constants, Utility Functions, and Data Importing

In [150]:
# Constants and Clients
GPT_MODEL = "gpt-3.5-turbo"
MAX_CHAR_LEN = 5000000
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

openai_client = OpenAI(api_key=OPENAI_API_KEY)

In [155]:
# Utility Functions
def cluster_list(messages: [dict]):
    """
    This function takes a piece of text and uses OpenAI's GPT-4 to rewrite it in a specified style.
    """
    openai.api_key = OPENAI_API_KEY

    try:
        response = openai_client.chat.completions.create(
            model=GPT_MODEL, messages=messages, temperature=0.6, max_tokens=500
        )
        return response.choices[0].text.split("\n")
    except Exception as e:
        return str(e)


def generate_text_with_style(prompt: str):
    """
    This function takes a piece of text and uses OpenAI's GPT-4 to rewrite it in a specified style.
    """
    openai.api_key = OPENAI_API_KEY

    try:
        response = openai_client.completions.create(
            model=GPT_MODEL + "-instruct",
            prompt=prompt,
            temperature=0.6,
            max_tokens=300,
        )
        return response.choices[0].text.strip()
    except Exception as e:
        return str(e)

In [152]:
# Data Importing
# Data is sourced from a personal project of mine you can find here:
# https://ucpd-incident-reporter-7cfdc3369124.herokuapp.com/
ucpd_feather_path = "data/fully_tokenized_ucpd_incidents.feather"
if os.path.isfile(ucpd_feather_path):
    ucpd_reports = pd.read_feather(ucpd_feather_path)
else:
    # WARNING: This step takes about 120 minutes, so don't run it unless you need to.
    ucpd_reports = pd.read_csv("data/incident_dump.csv")
    ucpd_reports["tokenized_text"] = ucpd_reports["comments"].apply(
        lucem_illud.word_tokenize
    )
    ucpd_reports["normalized_text"] = ucpd_reports["tokenized_text"].apply(
        lucem_illud.normalizeTokens
    )
    ucpd_reports.to_feather(ucpd_feather_path)
ucpd_reports.head(50)

,comments,disposition,incident,location,occurred,predicted_incident,reported,reported_date,ucpd_id,validated_address,validated_location,tokenized_text,normalized_text
0,A person was transported to Comer Hospital by ...,Closed,Mental Health Transport,6300 S. University Ave. (S. Woodlawn Ave. Char...,2/1/24 10:10 AM,None,2024-02-01T10:10:00-06:00,2024-02-01,24-00114,"6300 S UNIVERSITY AVE, CHICAGO, IL, 60637","41.78045407997166,-87.59732203570559","[A, person, was, transported, to, Comer, Hospi...","[person, transport, comer, hospital, cfd, ems,..."
1,Catalytic converter taken from a 2015 Toyota p...,Open,Theft from Motor Vehicle,1210 E. 57th St. (Public Way),1/31/24 to 2/1/24 6:00 PM to 7:00 AM,None,2024-02-01T10:18:00-06:00,2024-02-01,24-00115,"1210 E 57TH ST, CHICAGO, IL, 60637","41.79150658678615,-87.59602168542061","[Catalytic, converter, taken, from, a, 2015, T...","[catalytic, converter, take, toyota, park, str..."
2,Debit and credit cards taken from wallet in un...,Open,Theft,900 E. 57th St. (Knapp Center),2/1/24 9:50 AM to 4:30 PM,None,2024-02-01T17:54:00-06:00,2024-02-01,24-00116,"900 E 57TH ST, CHICAGO, IL, 60637","41.791423455510476,-87.60366291896175","[Debit, and, credit, cards, taken, from, walle...","[debit, credit, card, take, wallet, unattended..."
3,Boyfriend battered girlfriend in off-campus pr...,Open,Domestic Battery,6040 S. Harper Ave. (Apt. Building),2/1/24 2:45 PM,None,2024-02-01T15:45:00-06:00,2024-02-01,24-00117,"6040 S HARPER AVE, CHICAGO, IL, 60637","41.78472618578524,-87.58821677767634","[Boyfriend, battered, girlfriend, in, off, cam...","[boyfriend, batter, girlfriend, campus, privat..."
4,A known suspect entered the off-campus store a...,Referred,Information / Theft,1346 E. 53rd St. (Target),1/31/24 12:15 PM,None,2024-01-31T14:16:00-06:00,2024-01-31,2024-004118,"1346 E 53RD ST, CHICAGO, IL, 60615","41.79955044222366,-87.593062823983","[A, known, suspect, entered, the, off, campus,...","[know, suspect, enter, campus, store, take, me..."
5,Person found a social security card on the str...,Closed,Found Property,5800 S. Ellis Ave. (Public Way),1/31/24 12:10 PM,None,2024-01-31T12:10:00-06:00,2024-01-31,24-00113,"5800 S ELLIS AVE, CHICAGO, IL, 60637","41.78955246628657,-87.60133694890986","[Person, found, a, social, security, card, on,...","[person, find, social, security, card, street,..."
6,Window smashed and tools taken from van parked...,Referred,Information / Theft,1522 E. 59th St. (Public Way),1/30/24 8:00 AM to 9:15 AM,None,2024-01-30T09:35:00-06:00,2024-01-30,2024-003951,"1522 E 59TH ST, CHICAGO, IL, 60637","41.787940677385954,-87.5883629192693","[Window, smashed, and, tools, taken, from, van...","[window, smash, tool, take, van, park, street,..."
7,Unknown person(s) forced entry to victim's off...,CPD,Information / Burglary,S. Kenwood Ave. between E. 56th St. and E. 57t...,1/30/24 7:30 AM to 8:30 PM,None,2024-01-30T20:33:00-06:00,2024-01-30,2024-004004,"and South Kenwood Avenue, between, IL 60637","41.7912193,-87.6067356","[Unknown, person(s, forced, entry, to, victim,...","[unknown, person(s, force, entry, victim, camp..."
8,Unknown person attempted to enter victim's off...,CPD,Information / Attempted Burglary,S. Kimbark Ave. between E. 53rd St. and E. 54t...,1/30/24 8:15 PM,None,2024-01-30T20:46:00-06:00,2024-01-30,2024-004007,"between E. 53rd St. and, S. Kimbark Ave, Chica...","41.7980034,-87.5941627","[Unknown, person, attempted, to, enter, victim...","[unknown, person, attempt, enter, victim, camp..."
9,Victim walking in the crosswalk was struck by ...,Open,Traffic Crash / Hit and Run / Personal Injury,5500 S. Ingleside Ave. (Public Way),1/30/24 7:40 AM,None,2024-01-30T07:47:00-06:00,2024-01-30,24-00105,"5500 South Ingleside Avenue, Chicago, IL 60615...","41.7956509,-87.6030342","[Victim, walking, in, the, crosswalk, was, str...","[victim, walk, crosswalk, strike, unknown, veh..."


## <font color="red">*Exercise 1*</font>

<font color="red">As this week's challenging questions asks, we'd like you to 
think how LLM can help your final project.  Try to use the OpenAI API to analyze 
a small-sized dataset (Remember to monitor API use on your OpenAI account!). The 
data could a sample from the dataset you prepare for the final project or some 
others. If it's going be a conventional task like classification, compare and 
see how it could beat(or being defeated) by other algorithms you've learned from 
previous weeks. If it's a special task that you cannot find a learned algorithm 
to compare with, evaluate its performance on your own and try if you can improve 
by changing hyperparameters(see [here](https://platform.openai.com/docs/api-reference/chat/create)), the prompt, etc.

In [143]:
cluster_list(ucpd_reports["comments"].to_list()[:50])

"Missing required arguments; Expected either ('model' and 'prompt') or ('model', 'prompt' and 'stream') arguments to be given"

## <font color="red">*Exercise 2*</font>

<font color="red">Fine-tune an LLM. You can either use the model (llama-2-7b) in 
the example code or find another open-source LLM. You may use datasets provided 
by HuggingFace or a dataset you collect from somewhere else (for your final project). 
If the task happens to be the same as in exercise 1, You can choose to compare the 
performance between the OpenAI LLM and your fine-tuned LLM. You can also choose to 
compare the performance between the vanilla and the fine-tuned LLM.

## <font color="red">*Exercise 3*</font>
<font color="red">Use LLM to generate some data and compare the differences between model-generated 
data and actual data. This exercise should not be a repetition of exercise 1. You should 
focus more on analyzing language nuances, qualitatively or quantitatively. You should also 
notice how the choice of LLM has possibly impacted the language it uses.

## <font color="red">*Exercise 4*</font>

<font color="red">Compare how LLMs change their performance with different 
shots on your task. If the evaluation criterion is quantifiable, such as 
classification with ground truth labels, plot and show how accuracy changes. 
If the evaluation criterion cannot be easily quantified, such as the clarity 
of explaining a concept, use your imagination to do some comparison (for 
example, you can ask another LLM to rate its peer :)) If you find close-sourced 
LLM APIs pricey and are unsatisfied with responses from small-sized open-sourced 
LLMs, you can try large-sized LLMs (such as 70B version Llama-2) with Petals 
(see [here](https://colab.research.google.com/drive/1uCphNY7gfAUkdDrTx21dZZwCOUDCMPw8?usp=sharing) and [here](https://colab.research.google.com/drive/1Ervk6HPNS6AYVr3xVdQnY5a-TjjmLCdQ)).

## <font color="red">*Exercise 5*</font>

<font color="red">Using Actor - Critical method to improve an LLM's performance 
on your task or doing some experiments langauge style learning (For example, 
you can investigate how LLMs perceive different groups of people would write 
their dating profiles. This may serve as an opportunity to explore how LLMs 
semantically embed social groups and assess their appropriateness.)

I am going to attempt to write a message stating my passions for a dating
app based on the school I attended at 3 different ages.

In [156]:
# University of Texas at Austin
ut_base_prompt = (
    "I graduated from the University of Texas at Austin and in my free time I like to run, "
    "read science fiction, explore museums, and visit cocktail bars."
)

ut_prompt_25 = f"Rewrite the following text in 150 words using the style of a 25 year old male:\n\n{ut_base_prompt}"
ut_prompt_30 = f"Rewrite the following text in 150 words using the style of a 30 year old male:\n\n{ut_base_prompt}"
ut_prompt_35 = f"Rewrite the following text in 150 words using the style of a 35 year old male:\n\n{ut_base_prompt}"

In [157]:
ut_init_response_25 = generate_text_with_style(ut_prompt_25)
print(ut_init_response_25)

So, I just wrapped up my time at UT Austin and got that degree in the bag. Pretty stoked about it, gotta admit. When I'm not grinding away at my 9-5, I'm all about that runner's high and diving into some sci-fi reads. But let's be real, I can't be a total nerd all the time. I love hitting up museums and checking out all the dope exhibits. And let's not forget about my love for a good cocktail. Nothing beats unwinding with a fancy drink in hand. Cheers to that.


In [158]:
print(
    generate_text_with_style(
        "Critique and re-write this text with a focus on clarity, age-appropriate phrasing, "
        f"and the vocabulary expected from a University of Texas at Austin graduate:\n\n{ut_init_response_25}"
    )
)

I recently completed my studies at UT Austin and successfully obtained my degree. I am very excited and proud of this achievement. In my free time outside of my job, I enjoy running and immersing myself in science fiction literature. However, I also enjoy exploring museums and admiring their fascinating exhibits. And of course, I have a fondness for indulging in a well-crafted cocktail to relax and unwind. Here's to enjoying the finer things in life.


In [159]:
ut_init_response_30 = generate_text_with_style(ut_prompt_30)
print(ut_init_response_30)

After conquering the rigorous academic challenges at the University of Texas at Austin, I proudly don the cap and gown and entered the real world. When I'm not hustling at my job, you can find me pounding the pavement on my daily runs, getting lost in the pages of mind-bending science fiction, or immersing myself in the world of art and history at various museums. And let's not forget my love for a well-crafted cocktail at a classy bar - a perfect way to unwind after a long day. My passions may seem diverse, but they all bring a sense of balance and fulfillment to my life. Cheers to being a well-rounded 30-year-old man!


In [160]:
print(
    generate_text_with_style(
        "Critique and re-write this text with a focus on clarity, age-appropriate phrasing, "
        f"and the vocabulary expected from a University of Texas at Austin graduate:\n\n{ut_init_response_30}"
    )
)

After successfully completing the challenging academic journey at the University of Texas at Austin, I proudly wear my graduation attire and have entered the workforce. When I'm not working, you can find me running, reading thought-provoking science fiction, or exploring art and history at different museums. I also enjoy unwinding with a well-made cocktail at a sophisticated bar after a busy day. My interests may vary, but they all contribute to a well-balanced and fulfilling life. Here's to being a well-rounded 30-year-old!


In [161]:
ut_init_response_35 = generate_text_with_style(ut_prompt_35)
print(ut_init_response_35)

After earning my degree at the prestigious University of Texas at Austin, I've developed a love for running, indulging in science fiction literature, discovering new museums, and sipping on expertly crafted cocktails at trendy bars. These activities not only keep me physically and mentally fit, but also allow me to unwind and escape from the daily grind of adulting. I relish in the thrill of pushing my limits while running, getting lost in futuristic worlds through sci-fi novels, learning about history and culture through museum exhibits, and indulging in the finer things in life at cocktail bars. As a 35-year-old man, I prioritize self-care and making the most out of my free time. It's important to balance work and play, and these hobbies allow me to do just that.


In [162]:
print(
    generate_text_with_style(
        "Critique and re-write this text with a focus on clarity, age-appropriate phrasing, "
        f"and the vocabulary expected from a University of Texas at Austin graduate:\n\n{ut_init_response_35}"
    )
)

After completing my education at the esteemed University of Texas at Austin, I have developed a passion for various activities such as running, reading science fiction, visiting museums, and enjoying well-crafted cocktails at popular bars. These pastimes not only keep me physically and mentally healthy, but also provide an escape from the daily stresses of adulthood. I find joy in challenging my physical abilities while running, immersing myself in futuristic worlds through sci-fi literature, expanding my knowledge of history and culture at museums, and indulging in the finer things in life at cocktail bars. As a 35-year-old, I prioritize taking care of myself and maximizing my free time. Maintaining a balance between work and leisure is crucial, and these hobbies allow me to achieve that balance.


In [163]:
# University of North Texas
unt_base_prompt = "I graduated from the University of North Texas and in my free time I like to run, read science fiction, explore museums, and visit cocktail bars."

unt_prompt_25 = f"Rewrite the following text in 150 words using the style of a 25 year old male:\n\n{unt_base_prompt}"
unt_prompt_30 = f"Rewrite the following text in 150 words using the style of a 30 year old male:\n\n{unt_base_prompt}"
unt_prompt_35 = f"Rewrite the following text in 150 words using the style of a 35 year old male:\n\n{unt_base_prompt}"

In [164]:
unt_init_response_25 = generate_text_with_style(unt_prompt_25)
print(unt_init_response_25)

So, after four long years of grinding it out at the University of North Texas, I finally got that piece of paper that says I'm a college grad. And let me tell you, it feels pretty damn good. But when I'm not hustling at my 9-5, you can catch me hitting the pavement for a run, getting lost in some mind-bending science fiction, or checking out the latest exhibit at the museum. And let's not forget about my love for a good cocktail. You can find me posted up at the coolest bars in town, sipping on some fancy concoction and living my best life. Because hey, after all that hard work, I deserve to let loose a little. Cheers to adulthood, am I right?


In [165]:
print(
    generate_text_with_style(
        "Critique and re-write this text with a focus on clarity, age-appropriate phrasing, "
        f"and the vocabulary expected from a University of North Texas graduate:\n\n{unt_init_response_25}"
    )
)

After completing four years at the University of North Texas, I finally received my college degree. It's a great feeling of accomplishment. When I'm not working my 9-5 job, I enjoy going for runs, reading science fiction, and visiting the museum. I also have a passion for trying new cocktails, and you can often find me at the trendiest bars in town, indulging in a fancy drink and enjoying life. It's important to take some time to relax and have fun after working so hard. Here's to adulthood!


In [166]:
unt_init_response_30 = generate_text_with_style(unt_prompt_30)
print(unt_init_response_30)

After earning my degree from the University of North Texas, I've been keeping busy with my hobbies. I'm a big fan of hitting the pavement for a good run, getting lost in a sci-fi novel, checking out the latest exhibits at museums, and sipping on some top-notch cocktails at the hottest bars in town. Life's too short to waste it on mundane activities, so I make sure to make the most out of my free time. Plus, staying active and expanding my mind with some out-of-this-world stories keeps me feeling young and energized. So, you can catch me sweating it out on the trails, getting lost in a good book, or indulging in some fancy drinks at a classy bar. That's just how I roll.


In [167]:
print(
    generate_text_with_style(
        "Critique and re-write this text with a focus on clarity, age-appropriate phrasing, "
        f"and the vocabulary expected from a University of North Texas graduate:\n\n{unt_init_response_30}"
    )
)

Since graduating from the University of North Texas, I have been keeping myself occupied with my interests. I enjoy going for runs, reading science fiction books, visiting museums, and trying out new cocktails at popular bars. I believe that life is too valuable to waste on mundane tasks, so I always make the most of my free time. Staying physically active and expanding my mind with imaginative stories helps me feel youthful and invigorated. You can find me jogging on the trails, engrossed in a captivating novel, or treating myself to fancy drinks at an elegant bar. That's just my way of enjoying life.


In [168]:
unt_init_response_35 = generate_text_with_style(unt_prompt_35)
print(unt_init_response_35)

I'm a proud graduate of the esteemed University of North Texas, where I honed my intellectual prowess and earned my degree with flying colors. When I'm not busy with my job, which by the way, is killing it in the corporate world, I indulge in some of my favorite activities. Running is my go-to for staying fit and maintaining my alpha male physique. But let's not forget my love for sci-fi literature, it's my escape from reality and a way to exercise my imagination. On weekends, you can find me strolling through museums, absorbing the rich culture and history that surrounds us. And when the night falls, I like to unwind at the trendiest cocktail bars, sipping on a fine whiskey and discussing the latest world news with my buddies. That's just how I roll at 35, living life to the fullest.


In [169]:
print(
    generate_text_with_style(
        "Critique and re-write this text with a focus on clarity, age-appropriate phrasing, "
        f"and the vocabulary expected from a University of North Texas graduate:\n\n{unt_init_response_35}"
    )
)

As a proud graduate of the prestigious University of North Texas, I have developed my intellectual abilities and successfully completed my degree. When I am not occupied with my successful corporate job, I enjoy engaging in a variety of activities. To stay physically fit and maintain my strong physique, I regularly go for runs. In my free time, I am an avid reader of science fiction literature, which allows me to escape reality and exercise my imagination. On weekends, I like to visit museums to absorb the rich culture and history that surrounds us. As the night approaches, I unwind at popular cocktail bars, savoring a fine whiskey and discussing current world events with my friends. At the age of 35, I am living life to the fullest.


In [170]:
# University of Chicago
uc_base_prompt = "I graduated from the University of Chicago and in my free time I like to run, read science fiction, explore museums, and visit cocktail bars."

uc_prompt_25 = f"Rewrite the following text in 150 words using the style of a 25 year old male:\n\n{uc_base_prompt}"
uc_prompt_30 = f"Rewrite the following text in 150 words using the style of a 30 year old male:\n\n{uc_base_prompt}"
uc_prompt_35 = f"Rewrite the following text in 150 words using the style of a 35 year old male:\n\n{uc_base_prompt}"

In [171]:
uc_init_response_25 = generate_text_with_style(uc_prompt_25)
print(uc_init_response_25)

Yo, so I just graduated from the University of Chicago, no big deal. In my downtime, you can catch me hitting the pavement for a run, getting lost in some sick science fiction reads, checking out dope museums, and sipping on some fancy cocktails at the bar. I'm all about that balance, you feel me? Gotta keep the mind and body in check. But don't get it twisted, I'm not all about that boring stuff. I love to party and have a good time with my squad too. Life's too short to be serious all the time, am I right? Anyway, just trying to make the most out of my 20s and live it up while I can. #YOLO.


In [172]:
print(
    generate_text_with_style(
        "Critique and re-write this text with a focus on clarity, age-appropriate phrasing, "
        f"and the vocabulary expected from a University of Chicago graduate:\n\n{uc_init_response_25}"
    )
)

I recently graduated from the University of Chicago, and I'm pretty proud of that accomplishment. In my free time, you can usually find me going for a run, reading some captivating science fiction, exploring interesting museums, or enjoying a fancy cocktail at a bar. I believe in maintaining a balance between physical and mental well-being. However, don't mistake me for a boring person. I also enjoy socializing and having a good time with my friends. After all, life is too short to be serious all the time. I'm just trying to make the most of my 20s and have some fun while I can. #YOLO.


In [173]:
uc_init_response_30 = generate_text_with_style(uc_prompt_30)
print(uc_init_response_30)

After earning my degree from the prestigious University of Chicago, I've been keeping busy with my passions. Running is my go-to for staying fit and clearing my mind. But when I'm not pounding the pavement, you can find me engrossed in some mind-bending science fiction literature. And let's not forget about my love for expanding my knowledge through museum visits. But when the sun goes down, you can catch me sipping on some expertly crafted cocktails at the hottest bars in town. Life's too short to not indulge in the finer things, am I right? But don't get me wrong, I'm not all about indulgence. I take my career seriously and am always looking for new ways to challenge myself. Overall, I'm just a guy who knows what he wants and isn't afraid to go after it.


In [174]:
print(
    generate_text_with_style(
        "Critique and re-write this text with a focus on clarity, age-appropriate phrasing, "
        f"and the vocabulary expected from a University of Chicago graduate:\n\n{uc_init_response_30}"
    )
)

After graduating from the esteemed University of Chicago, I have been actively pursuing my interests. I prioritize running as a means of staying physically fit and maintaining mental clarity. In my free time, I also enjoy delving into thought-provoking science fiction literature. Additionally, I have a strong passion for expanding my knowledge through frequent visits to museums. When the day comes to a close, you can find me enjoying expertly crafted cocktails at the most popular bars in town. I believe life is meant to be enjoyed, and I make sure to indulge in the finer things. However, I am also dedicated to my career and continuously seek new challenges. Ultimately, I am a determined individual who is unafraid to pursue my desires.


In [175]:
uc_init_response_35 = generate_text_with_style(uc_prompt_35)
print(uc_init_response_35)

After earning my degree from the prestigious University of Chicago, I've found a sense of fulfillment in my professional career. But when I'm not crushing it in the office, you can find me indulging in a variety of hobbies. I love hitting the pavement for a good run, losing myself in the pages of a sci-fi novel, and immersing myself in the rich history of museums. And let's not forget my love for a finely crafted cocktail at a trendy bar. It's all about balance, am I right? But don't let my age fool you, I can still keep up with the young bucks on the dance floor. Life's too short to not enjoy it to the fullest. Cheers to that.


In [176]:
print(
    generate_text_with_style(
        "Critique and re-write this text with a focus on clarity, age-appropriate phrasing, "
        f"and the vocabulary expected from a University of Chicago graduate:\n\n{uc_init_response_35}"
    )
)

Since graduating from the esteemed University of Chicago, I have found fulfillment in my professional pursuits. When I am not occupied with work, I enjoy engaging in a variety of hobbies. I delight in going for a run, getting lost in the pages of a science fiction novel, and immersing myself in the rich history of museums. Additionally, I appreciate indulging in a well-crafted cocktail at a trendy bar. As they say, balance is key. Despite my age, I am still able to keep up with the younger crowd on the dance floor. After all, life is too short to not savor every moment. Here's to making the most of it.
